Make the import

In [1]:
import numpy as np
import mne
import myonset as myo

# Import EMG data

Set EMG file name

In [2]:
fname  = 'example_data.bdf'

Extract file name and create log file

Load raw data

In [3]:
raw = mne.io.read_raw_bdf(fname,preload=True)

Extracting EDF parameters from C:\Users\Laure_Spieser\Nextcloud\Documents\myonset_public\myonset\tutorials\example_data.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 118783  =      0.000 ...    58.000 secs...


# Extract events 

In [4]:
trigger_events = mne.find_events(raw, stim_channel='Status')

80 events found
Event IDs: [  1   2  10  11  12  20  21  22  57 100 200 254 255]


Put mne events in our events structure

In [5]:
events = myo.Events(sample=trigger_events[:,0], code=trigger_events[:,2], chan=[-1]*trigger_events.shape[0],\
                    sf=raw.info['sfreq'])

# Pre-process signal

In [6]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: EXG1, EXG2, EXG3, EXG4, Status
 chs: 4 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2018-02-26 08:24:27 UTC
 nchan: 5
 projs: []
 sfreq: 2048.0 Hz
>

EMG bipolar reference 

In [7]:
myo.use_mne.bipolar_ref(raw, ['EXG1','EXG3'], ['EXG2','EXG4'], new_ch=['EMG_1','EMG_2'], copy=False)
raw.info['ch_names']

EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=2, n_times=118784
    Range : 0 ... 118783 =      0.000 ...    58.000 secs
Ready.
Added the following bipolar channels:
EMG_1, EMG_2


['Status', 'EMG_1', 'EMG_2']

Filter EMG

In [8]:
raw = myo.use_mne.apply_filter(raw, ch_names=['EMG_1','EMG_2'],low_cutoff = 10)

Get pre-processed raw data

In [9]:
data = myo.use_mne.get_data_array(raw, ch_names=['EMG_1','EMG_2'])
data.shape

(2, 118784)

# Save data and events

In [10]:
np.save('example_data.npy',data)

In [11]:
events.to_csv('example_events.csv')